In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

Function to normalize images, but since images (0-255) can simply just divide by 255

In [ ]:
def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

def preprocess(row_entry, label):
    transform1 = row_entry.reshape((3,32,32))
    transform2 = np.transpose(transform1, [1,2,0])
    return transform2

## Test on 10000 training and 5000 test samples

In [ ]:
import numpy as np
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Method for reading in the "pickled" object images
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Method for preprocessing the data
def preprocess(row_entry, label):
    transform1 = row_entry.reshape((3,32,32))
    transform2 = np.transpose(transform1, [1,2,0])
    return transform2

# Read in the datasets 5 training batches and 1 test batch, each has 10,000 images
data_batch_1 = unpickle('data_batch_1')
data_batch_2 = unpickle('data_batch_2')
# data_batch_3 = unpickle('data_batch_3')
# data_batch_4 = unpickle('data_batch_4')
# data_batch_5 = unpickle('data_batch_5')
# test_batch = unpickle('test_batch')

# Each data_batch is a dictionary with the following items
# b'batch_label --> specifies which batch it is
# b'labels --> array of 10,000 labels 0-9 correspoding to the correct classification
# b'data --> 10,000 x 3072 array of uint8 pixels, each rows is a 32x32 image with the first 1024 entries being the red,
#            the second 1024 entries being the green, and the last 1024 entries being the blue

db1_labels = data_batch_1[b'labels']
db1_data = data_batch_1[b'data']
db2_labels = data_batch_2[b'labels']
db2_data = data_batch_2[b'data']
# db3_labels = data_batch_3[b'labels']
# db3_data = data_batch_3[b'data']
# db4_labels = data_batch_4[b'labels']
# db4_data = data_batch_4[b'data']
# db5_labels = data_batch_5[b'labels']
# db5_data = data_batch_5[b'data']
# db6_labels = test_batch[b'labels']
# db6_data = test_batch[b'data']
# tb_labels = test_batch[b'labels']
# tb_data = test_batch[b'data']

print(len(db1_data[0,:])) # --> first row
print(len(db1_data[:,0])) # -->first column

for i in range(0,10000):
    db1_data[i,:] = db1_data[i,:] / 255
    
for i in range(0,5000):
    db2_data[i,:] = db2_data[i,:] / 255
    
# def ridge_regression(A1, A2, A3, A4, A5, d1, d2, d3, d4, d5, T1, T2, y1, y2, lambdas):
def ridge_regression(A1, d1, T1, T2, y1, y2, lambdas):
    print("Running Ridge Regression")
#     A = np.vstack((A1, A2, A3, A4, A5)) #Training matrix
#     d = np.vstack((d1, d2, d3, d4, d5)) #Known classifiers
    A = A1 #Training matrix
    d = d1 #Known classifiers

    print(len(A[:,0]))
    print(len(A[0,:]))

    num_iterations = len(lambdas)
    training_errors = np.zeros(num_iterations)

    # Perform the training over all the different lambdas
    for lam in range(0, num_iterations):
        print("Iteration " + str(lam) + ", using lambda = " + str(lambdas[lam]))
        w = np.linalg.inv(A.T @ A + lambdas[lam] * np.identity(len(A[0,:]))) @ A.T @ d

        # Find the predictions for the first test set
        t_hat = T1 @ w
        error_count = 0

        # Record the number of errors
        for i in range(0, len(t_hat)):
            if abs(round(t_hat[i])) != y1[i]:
#                 if i < 5:
#                     print("y = " +str(y1[i]) + ", y_hat = "+str(abs(round(t_hat[i]))))
                error_count += 1
        
        training_errors[lam] = error_count
    
    # Determine which lambda gave the lowest error rates
    min_idx = 0
    min_error = 50000

    for i in range(0,num_iterations):
        if training_errors[i] < min_error:
            min_idx = i
            min_error = training_errors[i]
    

    # Use the selected lambda with the rest of the training data to get w
    lam = lambdas[min_idx]
    print("Lambda Chosen: " + str(lam))

    w = np.linalg.inv(A.T @ A + lam * np.identity(len(A[0,:]))) @ A.T @ d

    # Find the predictions for the second test set
    y_hat = T2 @ w
    error_count = 0

    # Record the number of errors
    for i in range(0, len(y_hat)):
        if abs(round(y_hat[i])) != y2[i]:
            if i < 5:
                print("y = " +str(y2[i]) + ", y_hat = "+str(abs(round(y_hat[i]))))
            error_count += 1

    # Calculate the errors and return them
    error_rate = error_count / len(y2)
    squared_error = np.linalg.norm(y_hat - abs(y2))**2

    return ([error_rate, squared_error])



# Logarithmic spaced lambdas
# TODO: how to determine the span of these?
lambdas = np.logspace(-6,np.log(20))
# lambdas = [0,0.25,0.5,0.75,1,2,4,8]

# [error_rate1, squared_error1] = ridge_regression(db2_data, db3_data, db4_data, db5_data, db6_data, 
#                                                     db2_labels, db3_labels, db4_labels, db5_labels, db6_labels, 
#                                                     db1_data[0:5000,:], db1_data[5000:10000,:], db1_labels[0:5000], 
#                                                     db1_labels[5000:10000], lambdas)

[error_rate1, squared_error1] = ridge_regression(db1_data, db1_labels, 
                                                    db2_data[0:2500,:], db2_data[2500:5000,:], db2_labels[0:2500], 
                                                    db2_labels[2500:5000], lambdas)
print("Ridge Regression Iteration 1")
print("Error Rate: " + str(round(error_rate1*100,3)) + ", Sqaured Error: " + str(round(squared_error1,3)))
print()

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 01:44:56
3072
10000
Running Ridge Regression
10000
3072
Iteration 0, using lambda = 1e-06
Iteration 1, using lambda = 1.52611187931549e-06
Iteration 2, using lambda = 2.3290174681878612e-06
Iteration 3, using lambda = 3.5543412253347817e-06
Iteration 4, using lambda = 5.424322367124196e-06
Iteration 5, using lambda = 8.278122801704955e-06
Iteration 6, using lambda = 1.2633341546114356e-05
Iteration 7, using lambda = 1.927989260897504e-05
Iteration 8, using lambda = 2.9423273142483782e-05
Iteration 9, using lambda = 4.4903206671088995e-05
Iteration 10, using lambda = 6.852731712010748e-05
Iteration 11, using lambda = 0.00010458035271461577
Iteration 12, using lambda = 0.00015960131862077925
Iteration 13, using lambda = 0.00024356946830158795
Iteration 14, using lambda = 0.00037171425901361105
Iteration 15, using lambda = 0.0005672775463916274
Iteration 16, using lambda = 0.0008657290024172074


## Full dataset, only 1 trial

In [ ]:
import numpy as np
from datetime import datetime

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# Method for reading in the "pickled" object images
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Method for preprocessing the data
def preprocess(row_entry, label):
    transform1 = row_entry.reshape((3,32,32))
    transform2 = np.transpose(transform1, [1,2,0])
    return transform2

# Read in the datasets 5 training batches and 1 test batch, each has 10,000 images
data_batch_1 = unpickle('data_batch_1')
data_batch_2 = unpickle('data_batch_2')
data_batch_3 = unpickle('data_batch_3')
data_batch_4 = unpickle('data_batch_4')
data_batch_5 = unpickle('data_batch_5')
test_batch = unpickle('test_batch')

# Each data_batch is a dictionary with the following items
# b'batch_label --> specifies which batch it is
# b'labels --> array of 10,000 labels 0-9 correspoding to the correct classification
# b'data --> 10,000 x 3072 array of uint8 pixels, each rows is a 32x32 image with the first 1024 entries being the red,
#            the second 1024 entries being the green, and the last 1024 entries being the blue

db1_labels = data_batch_1[b'labels']
db1_data = data_batch_1[b'data']
db2_labels = data_batch_2[b'labels']
db2_data = data_batch_2[b'data']
db3_labels = data_batch_3[b'labels']
db3_data = data_batch_3[b'data']
db4_labels = data_batch_4[b'labels']
db4_data = data_batch_4[b'data']
db5_labels = data_batch_5[b'labels']
db5_data = data_batch_5[b'data']
db6_labels = test_batch[b'labels']
db6_data = test_batch[b'data']
# tb_labels = test_batch[b'labels']
# tb_data = test_batch[b'data']

print(len(db1_data[0,:])) # --> first row
print(len(db1_data[:,0])) # -->first column

for i in range(0,10000):
    db1_data[i,:] = db1_data[i,:] / 255
    db2_data[i,:] = db2_data[i,:] / 255
    db3_data[i,:] = db3_data[i,:] / 255
    db4_data[i,:] = db4_data[i,:] / 255
    db5_data[i,:] = db5_data[i,:] / 255
    db6_data[i,:] = db6_data[i,:] / 255


###################################################################################################
#
# Ridge Regression
#
def ridge_regression(A1, A2, A3, A4, A5, d1, d2, d3, d4, d5, T1, T2, y1, y2, lambdas):
# def ridge_regression(A1, d1, T1, T2, y1, y2, lambdas):
    print("Running Ridge Regression")
    A = np.vstack((A1, A2, A3, A4, A5)) #Training matrix
    d = np.vstack((d1, d2, d3, d4, d5)) #Known classifiers
#     A = A1 #Training matrix
#     d = d1 #Known classifiers

    print(len(A[:,0]))
    print(len(A[0,:]))

    num_iterations = len(lambdas)
    training_errors = np.zeros(num_iterations)

    # Perform the training over all the different lambdas
    for lam in range(0, num_iterations):
        print("Iteration " + str(lam) + ", using lambda = " + str(lambdas[lam]))
        w = np.linalg.inv(A.T @ A + lambdas[lam] * np.identity(len(A[0,:]))) @ A.T @ d

        # Find the predictions for the first test set
        t_hat = T1 @ w
        error_count = 0

        # Record the number of errors
        for i in range(0, len(t_hat)):
            if abs(round(t_hat[i])) != y1[i]:
#                 if i < 5:
#                     print("y = " +str(y1[i]) + ", y_hat = "+str(abs(round(t_hat[i]))))
                error_count += 1
        
        training_errors[lam] = error_count
    
    # Determine which lambda gave the lowest error rates
    min_idx = 0
    min_error = 50000

    for i in range(0,num_iterations):
        if training_errors[i] < min_error:
            min_idx = i
            min_error = training_errors[i]
    

    # Use the selected lambda with the rest of the training data to get w
    lam = lambdas[min_idx]
    print("Lambda Chosen: " + str(lam))

    w = np.linalg.inv(A.T @ A + lam * np.identity(len(A[0,:]))) @ A.T @ d

    # Find the predictions for the second test set
    y_hat = T2 @ w
    error_count = 0

    # Record the number of errors
    for i in range(0, len(y_hat)):
        if abs(round(y_hat[i])) != y2[i]:
            if i < 5:
                print("y = " +str(y2[i]) + ", y_hat = "+str(abs(round(y_hat[i]))))
            error_count += 1

    # Calculate the errors and return them
    error_rate = error_count / len(y2)
    squared_error = np.linalg.norm(y_hat - abs(y2))**2

    return ([error_rate, squared_error])



# Logarithmic spaced lambdas
# TODO: how to determine the span of these?
# lambdas = np.logspace(-6,np.log(20))
lambdas = [0,0.25,0.5,0.75,1,2,4,8]

[error_rate1, squared_error1] = ridge_regression(db2_data, db3_data, db4_data, db5_data, db6_data, 
                                                    db2_labels, db3_labels, db4_labels, db5_labels, db6_labels, 
                                                    db1_data[0:5000,:], db1_data[5000:10000,:], db1_labels[0:5000], 
                                                    db1_labels[5000:10000], lambdas)

# [error_rate1, squared_error1] = ridge_regression(db2_data[0:1000,:], db2_labels[0:1000], 
#                                                     db1_data[0:500,:], db1_data[500:1000,:], db1_labels[0:500], 
#                                                     db1_labels[500:1000], lambdas)
print("Ridge Regression Iteration 1")
print("Error Rate: " + str(round(error_rate1*100,3)) + ", Sqaured Error: " + str(round(squared_error1,3)))
print()

now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 00:21:35
3072
10000
Running Ridge Regression
50000
3072
Iteration 0, using lambda = 0


## Sample Test data to verify algorithm

In [17]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
in_data = loadmat('face_emotion_data.mat')

# print([key for key in in_data])

X = in_data['X'] #mxn matric of face features
y = in_data['y'] #mx1 vector of labels, 1 = happy and -1 = angry

# print(np.matrix.round(X,3))
# print(len(X))

def ridge_regression(A1, A2, A3, A4, A5, A6, d1, d2, d3, d4, d5, d6, T1, y1, T2, y2, lambdas):
# def ridge_regression(A1, d1, T1, T2, y1, y2, lambdas):
    print("Running Ridge Regression")
    A = np.vstack((A1,A2,A3,A4,A5,A6)) #Training matrix
    d = np.vstack((d1,d2,d3,d4,d5,d6)) #Known results 
#     print(len(A[0,:]))
#     print(len(A[:,0]))
#     d = np.vstack((d1, d2, d3, d4, d5)) #Known classifiers
#     print(len(d[0,:]))
#     print(len(d[:,0]))
#     A = A1 #Training matrix
#     d = d1 #Known classifiers

#     print(len(A[:,0]))
#     print(len(A[0,:]))

    num_iterations = len(lambdas)
    training_errors = np.zeros(num_iterations)

    # Perform the training over all the different lambdas
    for lam in range(0, num_iterations):
#         print("Using lambda = " + str(lambdas[lam]))
        w1 = np.linalg.inv(A.T @ A + lambdas[lam] * np.identity(len(A[0,:]))) 
        w = w1 @ A.T @ d

        # Find the predictions for the first test set
        t_hat = np.sign(T1 @ w)
        error_count = 0
        
#         print(len(t_hat[0,:]))
#         print(len(t_hat[:,0]))
#         print(len(y1[0,:]))
#         print(len(y1[:,0]))

        # Record the number of errors
        for i in range(0, len(t_hat)):
            if t_hat[i] != y1[i]:
                error_count += 1
        
        training_errors[lam] = error_count
    
    # Determine which lambda gave the lowest error rates
    min_idx = 0
    min_error = 50000

    for i in range(0,num_iterations):
        if training_errors[i] < min_error:
            min_idx = i
            min_error = training_errors[i]
    

    # Use the selected lambda with the rest of the training data to get w
    lam = lambdas[min_idx]

    w = np.linalg.inv(A.T @ A + lam * np.identity(len(A[0,:]))) @ A.T @ d

    # Find the predictions for the second test set
    y_hat = np.sign(T2 @ w)
    error_count = 0

    # Record the number of errors
    for i in range(0, len(y_hat)):
        if y_hat[i] != y2[i]:
            error_count += 1

    # Calculate the errors and return them
    error_rate = error_count / len(y2)
    squared_error = np.linalg.norm(y_hat - y2)**2

    return ([error_rate, squared_error])


X1 = X[0:16,:]
X2 = X[16:32,:]
X3 = X[32:48,:]
X4 = X[48:64,:]
X5 = X[64:80,:]
X6 = X[80:96,:]
X7 = X[96:112,:]
X8 = X[112:128,:]

Xset = [X1,X2,X3,X4,X5,X6,X7,X8]

y1 = y[0:16]
y2 = y[16:32]
y3 = y[32:48]
y4 = y[48:64]
y5 = y[64:80]
y6 = y[80:96]
y7 = y[96:112]
y8 = y[112:128]

yset = [y1,y2,y3,y4,y5,y6,y7,y8]

lambdas = [0,0.25,0.5,0.75,1,2,4]

SVD_error = 0
RR_error = 0
count = 0
for i in range(0,8):
    for j in range(0,8):
        if i != j:
            count += 1
            T1 = None
            T2 = None
            T3 = None
            T4 = None
            T5 = None
            T6 = None
            y1 = None
            y2 = None
            y3 = None
            y4 = None
            y5 = None
            y6 = None
            
            for l in range(0,9):
                if l != i and l != j and T1 is None:
                    T1 = Xset[l]
                    r1 = yset[l]
                elif l != i and l != j and T2 is None:
                    T2 = Xset[l]
                    r2 = yset[l]
                elif l != i and l != j and T3 is None:
                    T3 = Xset[l]
                    r3 = yset[l]
                elif l != i and l != j and T4 is None:
                    T4 = Xset[l]
                    r4 = yset[l]
                elif l != i and l != j and T5 is None:
                    T5 = Xset[l]
                    r5 = yset[l]
                elif l != i and l != j and T6 is None:
                    T6 = Xset[l]
                    r6 = yset[l]
                    

            [error_rate1, squared_error1] = ridge_regression(T1,T2,T3,T4,T5,T6,r1,r2,r3,r4,r5,r6,Xset[i],yset[i],Xset[j],yset[j], lambdas)
            
            print("Ridge Regression Iteration")
            print("Error Rate: " + str(round(error_rate1*100,3)) + ", Sqaured Error: " + str(round(squared_error1,3)))
            
# print("Total Truncated SVD Error: " + str(round(SVD_error/56*100,3)) +"%")
# print("Total Ridge Regression Error: " + str(round(RR_error/56*100,3)) +"%")
# print(count)

Running Ridge Regression
Ridge Regression Iteration
Error Rate: 6.25, Sqaured Error: 4.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 12.5, Sqaured Error: 8.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 6.25, Sqaured Error: 4.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 0.0, Sqaured Error: 0.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 6.25, Sqaured Error: 4.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 6.25, Sqaured Error: 4.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 0.0, Sqaured Error: 0.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 12.5, Sqaured Error: 8.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 12.5, Sqaured Error: 8.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 0.0, Sqaured Error: 0.0
Running Ridge Regression
Ridge Regression Iteration
Error Rate: 0.0, Sqaured Error: 0.0
Running Ridge Regression
